### setup

In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np
import datetime
import random
import json

In [3]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [6]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [7]:
scope = ["https://spreadsheets.google.com/feeds"]
# credentials = ServiceAccountCredentials.from_json_keyfile_name(
#    "jupyter-integration-credentials-redistricting.json", scope
# )
# gc = gspread.authorize(credentials)

source: https://twitter.com/ddiamond/status/1555969337068081154<br>
data series:<br>
'SMU27000006562110001A' [physicians office](https://fred.stlouisfed.org/series/SMU27000006562110001A)<br>
Not available in MN (outpatient care centers)<br>
'SMU27000006562200001A' [hospitals](https://fred.stlouisfed.org/series/SMU27000006562200001A)<br>
'SMU27000006562300001A' [nursing homes](https://fred.stlouisfed.org/series/SMU27000006562300001A)<br>

### read in bls data from FRED

Physicians offices

In [8]:
with urllib.request.urlopen(
    "https://api.stlouisfed.org/fred/series/observations?series_id=SMU27000006562110001A&api_key=b98b54e0ff7221aca6e1b161f5b12435&file_type=json"
) as url:
    data = json.loads(url.read().decode())

In [9]:
dataframes = []
for d in data["observations"]:
    dataframes.append(pd.DataFrame(d, index=[0]))

In [10]:
physicians_office = pd.concat(dataframes).reset_index(drop=True)

Hospitals

In [11]:
with urllib.request.urlopen(
    "https://api.stlouisfed.org/fred/series/observations?series_id=SMU27000006562200001A&api_key=b98b54e0ff7221aca6e1b161f5b12435&file_type=json"
) as url:
    data = json.loads(url.read().decode())

In [12]:
dataframes = []
for d in data["observations"]:
    dataframes.append(pd.DataFrame(d, index=[0]))

In [13]:
hospitals = pd.concat(dataframes).reset_index(drop=True)

Nursing homes

In [14]:
with urllib.request.urlopen(
    "https://api.stlouisfed.org/fred/series/observations?series_id=SMU27000006562300001A&api_key=b98b54e0ff7221aca6e1b161f5b12435&file_type=json"
) as url:
    data = json.loads(url.read().decode())

In [15]:
dataframes = []
for d in data["observations"]:
    dataframes.append(pd.DataFrame(d, index=[0]))

In [16]:
nursing_homes = pd.concat(dataframes).reset_index(drop=True)

### clean dfs

In [17]:
physicians_office = physicians_office.drop(columns=["realtime_start", "realtime_end"])
nursing_homes = nursing_homes.drop(columns=["realtime_start", "realtime_end"])
hospitals = hospitals.drop(columns=["realtime_start", "realtime_end"])

In [18]:
physicians_office["type"] = "Physicians' offices"
nursing_homes["type"] = "Nursing homes"
hospitals["type"] = "Hospitals"

#### combining each venue into single df and changing the types of variables

In [19]:
df_list = [physicians_office, nursing_homes, hospitals]

In [20]:
for df in df_list:
    df["date"] = pd.to_datetime(df["date"])
    df.sort_values("date", ascending=True)
    df["value"] = df["value"].astype(float)
    df["type"] = df["type"].astype(str)
    df["pct_change"] = df["value"].pct_change()

In [21]:
combined = pd.concat(df_list)

### charting

In [22]:
alt.Chart(combined[combined["date"] >= "2020-01-01"]).mark_line().encode(
    x=alt.X("date"), y=alt.Y("value"), color=alt.Color("type")
)

alt.Chart(...)

In [41]:
combined.dtypes

date          datetime64[ns]
value                float64
type                  object
pct_change           float64
year                  object
dtype: object

In [40]:
combined["year"] = combined["date"].dt.year.astype(str)

In [87]:
(
    alt.Chart(
        combined[combined["date"] >= "2010-01-01"],
        padding={"left": -22, "top": 0, "right": 0, "bottom": 0},
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "year:O",
            title="",
            axis=alt.Axis(
                tickCount=3, values=["2010", "2015", "2020"], labelColor="#1a1a1a"
            ),
        ),
        y=alt.Y(
            "pct_change",
            title="",
            axis=alt.Axis(format="%", labelColor="#1a1a1a", tickCount=6),
        ),
        color=alt.condition(
            alt.datum.pct_change > 0, alt.value("#00D4D8"), alt.value("#D95F1A")
        ),
        facet=alt.Facet(
            "type:O",
            title="",
            columns=3,
            header=alt.Header(labelFontSize=15, labelFont="Summit Sans"),
        ),
    )
    .properties(width=270, height=300)
)

alt.Chart(...)

In [90]:
(
    alt.Chart(
        combined[combined["date"] >= "2010-01-01"],
        padding={"left": -22, "top": 0, "right": 0, "bottom": 0},
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "year:O",
            title="",
            axis=alt.Axis(
                tickCount=3, values=["2010", "2015", "2020"], labelColor="#1a1a1a"
            ),
        ),
        y=alt.Y(
            "pct_change",
            title="",
            axis=alt.Axis(format="%", labelColor="#1a1a1a", tickCount=6),
        ),
        color=alt.condition(
            alt.datum.pct_change > 0, alt.value("#00D4D8"), alt.value("#D95F1A")
        ),
        facet=alt.Facet(
            "type:O",
            title="",
            columns=3,
            header=alt.Header(labelFontSize=15, labelFont="Summit Sans"),
        ),
    )
    .properties(width=150, height=250)
)

alt.Chart(...)

In [93]:
(
    alt.Chart(
        combined[combined["date"] >= "2010-01-01"],
        padding={"left": -22, "top": 0, "right": 0, "bottom": 0},
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "year:O",
            title="",
            axis=alt.Axis(
                tickCount=3, values=["2010", "2015", "2020"], labelColor="#1a1a1a"
            ),
        ),
        y=alt.Y(
            "pct_change",
            title="",
            axis=alt.Axis(format="%", labelColor="#1a1a1a", tickCount=6),
        ),
        color=alt.condition(
            alt.datum.pct_change > 0, alt.value("#00D4D8"), alt.value("#D95F1A")
        ),
        facet=alt.Facet(
            "type:O",
            title="",
            columns=1,
            header=alt.Header(labelFontSize=15, labelFont="Summit Sans"),
        ),
    )
    .properties(width=300, height=250)
)

alt.Chart(...)

### look at change on their own scales

In [24]:
alt.Chart(hospitals[hospitals["date"] >= "2010-01-01"]).mark_line().encode(
    x=alt.X("date"), y=alt.Y("value")
).properties(width=400, height=200)

alt.Chart(...)

In [25]:
alt.Chart(nursing_homes[nursing_homes["date"] >= "2010-01-01"]).mark_line().encode(
    x=alt.X("date"), y=alt.Y("value")
).properties(width=400, height=200)

alt.Chart(...)

In [26]:
alt.Chart(
    physicians_office[physicians_office["date"] >= "2010-01-01"]
).mark_line().encode(x=alt.X("date"), y=alt.Y("value")).properties(
    width=400, height=200
)

alt.Chart(...)